[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/ronghuaxueleng/colab-stable-diffusion-webui/blob/main/stable_diffusion_webui_colab_zh_without_model.ipynb)


# Stable Diffusion WebUI Colab 简体中文

### 使用流程

1. 到**下载模型**, **Extensions**, **其他设置**进行设置
2. 左上角工具列 > 执行阶段 > 变更执行阶段类型 > 硬件加速器改为 GPU
3. 首次执行步骤：左上角工具列 > 执行阶段 > 全部执行
   非首次执行步骤：**前置步骤**->**Extensions**->**其他设置**->**启动WebUI**
4. 到**启动 WebUI** 点进 public URL

### 参考资料
https://colab.research.google.com/drive/1lekLF7iib6M1R-NCylS0VMTF4wve-XuV

### 更新时间
2023/04/20

## 1 - 前置步骤

In [ ]:
# @title 1.1 下载 stable-diffusion-webui
%cd /content
webui_version = "AUTOMATIC1111(03/29)" # @param ["AUTOMATIC1111(03/29)", "AUTOMATIC1111(03/25)", "anapnoe(最新版本)", "其他(填写repository_url和webui_branch)"]
repository_url = ""  # @param {type: "string"}
webui_branch = ""  # @param {type: "string"}
# @markdown ---
# @markdown - https://github.com/d930065/SDWCT
# @markdown - https://github.com/Linaqruf/anapnoe-ui

if webui_version == "AUTOMATIC1111(03/29)":
    repository_url = "https://github.com/d930065/SDWCT"
    webui_branch = "22bcc7be428c94e9408f589966c2040187245d81"
elif webui_version == "AUTOMATIC1111(03/25)":
    repository_url = "https://github.com/d930065/SDWCT"
    webui_branch = "a9eab236d7e8afa4d6205127904a385b2c43bb24"
elif webui_version == "anapnoe(最新版本)":
    repository_url = "https://github.com/Linaqruf/anapnoe-ui"
    webui_branch = "master"

! git clone {repository_url} SDWCT
! git clone https://github.com/d930065/sd-webui-colab-styles-patch

%cd /content/SDWCT
! git checkout -f {webui_branch}

%cd /content/SDWCT/modules
! patch --binary -i /content/sd-webui-colab-styles-patch/styles.patch

In [ ]:
# @title 1.2 建立云端资料夹
%cd /content
# @markdown 云端资料夹名称
data_dir_name = "Stable_Diffusion_WebUI_Colab_zh_CN"  # @param {type:"string"}
data_dir_base = "/content/drive/MyDrive"
data_dir = f"{data_dir_base}/{data_dir_name}"

import os

drive_path = "/content/drive"
from google.colab import drive

try:
    drive.mount(drive_path, force_remount=True)
    os.makedirs(data_dir, exist_ok=True)
except:
    print('Mount Error')
    raise

if os.path.exists(f"{data_dir}/script.pre.sh"):
    ! chmod +x {data_dir}/script.pre.sh
    ! {data_dir}/script.pre.sh

models_path = f"{data_dir}/models"
output_path = f"{data_dir}/outputs"
config_path = f"{data_dir}/config"
scripts_path = f"{data_dir}/scripts"
extensions_file_path = f"{data_dir}/extensions.txt"

os.makedirs(models_path, exist_ok=True)
os.makedirs(output_path, exist_ok=True)
os.makedirs(config_path, exist_ok=True)
os.makedirs(scripts_path, exist_ok=True)
os.makedirs(f"{models_path}/Stable-diffusion", exist_ok=True)
os.makedirs(f"{models_path}/VAE", exist_ok=True)
os.makedirs(f"{models_path}/embeddings", exist_ok=True)
os.makedirs(f"{models_path}/hypernetworks", exist_ok=True)
os.makedirs(f"{models_path}/Lora", exist_ok=True)
os.makedirs(f"{models_path}/ControlNet", exist_ok=True)

! cp -Rf SDWCT/scripts/* {scripts_path}
! rm -Rf SDWCT/scripts && ln -s {scripts_path} SDWCT/scripts

for dir in os.listdir(models_path):
    if dir == "embeddings":
        ! rm -Rf SDWCT/embeddings && ln -s {models_path}/embeddings SDWCT/embeddings
    elif dir != "ControlNet":
        ! rm -Rf SDWCT/models/{dir} && ln -s {models_path}/{dir} SDWCT/models/{dir}

! rm -Rf SDWCT/outputs && ln -s {data_dir}/outputs SDWCT/outputs

for filename in ["config.json", "ui-config.json", "styles.csv", "extensions.txt"]:
    ! rm -f SDWCT/{filename}
    filepath = f"{config_path}/{filename}"
    if not os.path.exists(filepath):
        if filename.endswith(".json"):
            with open(filepath, mode="w") as f:
                f.write("{}")
        else:
            ! touch {config_path}/{filename}
    ! ln -s {config_path}/{filename} SDWCT/{filename}

In [ ]:
# @title 1.3 设置python环境
!wget -O mini.sh https://repo.anaconda.com/miniconda/Miniconda3-latest-Linux-x86_64.sh
!chmod +x mini.sh
!bash ./mini.sh -b -f -p /usr/local
#!conda search "^python$"
!conda install -y python=3.10.6
!which python
!python --version
!pip install -q torch==1.13.1+cu116 torchvision==0.14.1+cu116 torchaudio==0.13.1 torchtext==0.14.1 torchdata==0.5.1 --extra-index-url https://download.pytorch.org/whl/cu116 -U
!pip install -q xformers==0.0.16 triton==2.0.0 -U

## 2 - Extensions

In [ ]:
# @title 2.1 下载 Extensions
locon = True # @param {type:"boolean"}
localization_zh_CN = True # @param {type:"boolean"}
bilingual_localization = False # @param {type:"boolean"}
tag_complete = True # @param {type:"boolean"}
image_browser = True # @param {type:"boolean"}
civitai_browser = True # @param {type:"boolean"}
Civitai_Helper = True # @param {type:"boolean"}
cutoff = False # @param {type:"boolean"}
control_net = False # @param {type:"boolean"}
openpose_editor = False # @param {type:"boolean"}
posex = False # @param {type:"boolean"}
depth_lib = False # @param {type:"boolean"}
latent_couple = False # @param {type:"boolean"}
composable_lora = False # @param {type:"boolean"}
lora_block_weight = False # @param {type:"boolean"}
extension_urls = ""  # @param {type:"string"}

# @markdown ---
# @markdown - [[AI绘图] Stable Diffusion WebUI Colab TW Extensions整理](https://home.gamer.com.tw/artwork.php?sn=5678466)
# @markdown - 网址之间用英文逗号隔开。例：`url1`, `url2`, `url3`
# @markdown - 如果使用WebUI內建方法下载 Extension，仍然要执行 `下载 Extensions` 以记录下载的 Extension。
# @markdown - 如果不想再下载某个 Extension，请删除 `config/extensions.txt` 上对应的 GitHub 网址。


extensions_pair = [   
            (locon, "https://github.com/KohakuBlueleaf/a1111-sd-webui-locon"),
            (localization_zh_CN, "https://github.com/dtlnor/stable-diffusion-webui-localization-zh_CN"),
            (bilingual_localization, "https://github.com/journey-ad/sd-webui-bilingual-localization"),
            (tag_complete, "https://github.com/DominikDoom/a1111-sd-webui-tagcomplete"),
            (image_browser, "https://github.com/AlUlkesh/stable-diffusion-webui-images-browser"),
            (civitai_browser, "https://github.com/camenduru/sd-civitai-browser"),
            (Civitai_Helper, "https://github.com/butaixianran/Stable-Diffusion-Webui-Civitai-Helper"),
            (cutoff, "https://github.com/hnmr293/sd-webui-cutoff"),
            (control_net, "https://github.com/Mikubill/sd-webui-controlnet"),
            (openpose_editor, "https://github.com/fkunn1326/openpose-editor"),
            (posex, "https://github.com/hnmr293/posex"),
            (depth_lib, "https://github.com/jexom/sd-webui-depth-lib"),
            (latent_couple, "https://github.com/ashen-sensored/stable-diffusion-webui-two-shot"),
            (composable_lora, "https://github.com/opparco/stable-diffusion-webui-composable-lora"),
            (lora_block_weight,"https://github.com/hako-mikan/sd-webui-lora-block-weight")
          ]

with open('/content/SDWCT/extensions.txt', 'r') as f:
    extensions_txt = f.read()
extensions_txt = list(map(str.strip, extensions_txt.split("\n")))

extension_urls = list(map(str.strip, extension_urls.split(",")))

extensions = []
for url in extensions_txt:
    if url != '' and url not in extensions:
        extensions.append(url)   
for name, url in extensions_pair:
    if name == True and url not in extensions:
        extensions.append(url)
for url in extension_urls:
    if url != '' and url not in extensions:
        extensions.append(url)

import os

if extensions != []:
    %cd /content/SDWCT/extensions
    for extension in extensions:
        extension_name, _ = os.path.splitext(extension.split("/")[-1])
        if not os.path.exists(extension_name):
            ! git clone {extension}

%cd /content/SDWCT/extensions
if os.path.exists("sd-webui-controlnet") and not os.path.islink("sd-webui-controlnet/models"):
    ! cp sd-webui-controlnet/models/cldm_v15.yaml {models_path}/ControlNet/cldm_v15.yaml
    ! cp sd-webui-controlnet/models/cldm_v21.yaml {models_path}/ControlNet/cldm_v21.yaml
    ! cp sd-webui-controlnet/models/image_adapter_v14.yaml {models_path}/ControlNet/t2iadapter_depth_sd14v1.yaml
    ! cp sd-webui-controlnet/models/image_adapter_v14.yaml {models_path}/ControlNet/t2iadapter_keypose_sd14v1.yaml
    ! cp sd-webui-controlnet/models/image_adapter_v14.yaml {models_path}/ControlNet/t2iadapter_openpose_sd14v1.yaml
    ! cp sd-webui-controlnet/models/image_adapter_v14.yaml {models_path}/ControlNet/t2iadapter_seg_sd14v1.yaml
    ! cp sd-webui-controlnet/models/sketch_adapter_v14.yaml {models_path}/ControlNet/t2iadapter_canny_sd14v1.yaml
    ! cp sd-webui-controlnet/models/sketch_adapter_v14.yaml {models_path}/ControlNet/t2iadapter_sketch_sd14v1.yaml
    ! cp sd-webui-controlnet/models/t2iadapter_color_sd14v1.yaml {models_path}/ControlNet/t2iadapter_color_sd14v1.yaml
    ! cp sd-webui-controlnet/models/t2iadapter_style_sd14v1.yaml {models_path}/ControlNet/t2iadapter_style_sd14v1.yaml
    ! rm -Rf sd-webui-controlnet/models && ln -s {models_path}/ControlNet sd-webui-controlnet/models

if os.path.exists("a1111-sd-webui-tagcomplete"):
    ! git clone -b tags https://github.com/ronghuaxueleng/colab-stable-diffusion-webui.git my_tags0424
    ! cp -rf my_tags0424/* ./a1111-sd-webui-tagcomplete/tags
    ! rm -rf my_tags0424
    ! ls ./a1111-sd-webui-tagcomplete/tags

%cd /content/SDWCT/extensions
! echo -n '' > /content/SDWCT/extensions.txt
for dir in next(os.walk("/content/SDWCT/extensions"))[1]:
    %cd {dir}
    if os.path.exists(".git"):
        ! git config --get remote.origin.url >> /content/SDWCT/extensions.txt
    %cd ..

In [ ]:
# @title 2.2 下载 ControlNet 模型
# @markdown ControlNet
control_canny = False # @param {type:"boolean"}
control_depth = False # @param {type:"boolean"}
control_hed = False # @param {type:"boolean"}
control_mlsd = False # @param {type:"boolean"}
control_normal = False # @param {type:"boolean"}
control_openpose = True # @param {type:"boolean"}
control_scribble = False # @param {type:"boolean"}
control_seg = False # @param {type:"boolean"}
# @markdown T2I Adapter
t2iadapter_canny = False # @param {type:"boolean"}
t2iadapter_color = False # @param {type:"boolean"}
t2iadapter_depth = False # @param {type:"boolean"}
t2iadapter_keypose = False # @param {type:"boolean"}
t2iadapter_openpose = False # @param {type:"boolean"}
t2iadapter_seg = False # @param {type:"boolean"}
t2iadapter_sketch = False # @param {type:"boolean"}
t2iadapter_style = False # @param {type:"boolean"}
# @markdown ---
# @markdown - 除了利用Colab下载模型，推荐使用云端硬盘以节省空间。
# @markdown  - [[AI绘图] WebUI Colab玩家小技巧：使用使用云端硬盘以节省空间](https://home.gamer.com.tw/artwork.php?sn=5677784)
# @markdown - 多重 ControlNet 请到 Settings > ControlNet > Multi ControlNet 设置
# @markdown - 参考资料
# @markdown  - https://github.com/Mikubill/sd-webui-controlnet
# @markdown  - https://github.com/lllyasviel/ControlNet
# @markdown  - https://github.com/TencentARC/T2I-Adapter
# @markdown  - [Stable diffusion ControlNet使用心得](https://home.gamer.com.tw/artwork.php?sn=5662905)

import os

control_net_models = []
t2iadapter_models = []

if control_canny == True:
    control_net_models.append("canny")
if control_depth == True:
    control_net_models.append("depth")
if control_hed == True:
    control_net_models.append("hed")
if control_mlsd == True:
    control_net_models.append("mlsd")
if control_normal == True:
    control_net_models.append("normal")
if control_openpose == True:
    control_net_models.append("openpose")
if control_scribble == True:
    control_net_models.append("scribble")
if control_seg == True:
    control_net_models.append("seg")

if t2iadapter_canny == True:
    t2iadapter_models.append("canny")
if t2iadapter_color == True:
    t2iadapter_models.append("color")
if t2iadapter_depth == True:
    t2iadapter_models.append("depth")
if t2iadapter_keypose == True:
    t2iadapter_models.append("keypose")
if t2iadapter_openpose == True:
    t2iadapter_models.append("openpose")
if t2iadapter_seg == True:
    t2iadapter_models.append("seg")
if t2iadapter_sketch == True:
    t2iadapter_models.append("sketch")
if t2iadapter_style == True:
    t2iadapter_models.append("style")

%cd /content/SDWCT/extensions

if os.path.exists('sd-webui-controlnet'):
    %cd sd-webui-controlnet/models
    for control_net_model in control_net_models:
        ! wget -nc --content-disposition https://huggingface.co/webui/ControlNet-modules-safetensors/resolve/main/control_{control_net_model}-fp16.safetensors
    for t2iadapter_model in t2iadapter_models:
        ! wget -nc --content-disposition https://huggingface.co/TencentARC/T2I-Adapter/resolve/main/models/t2iadapter_{t2iadapter_model}_sd14v1.pth        

In [ ]:
# @title 2.3 下载档案到指定文件夹
target_dir = "" # @param {type:"string"} 
file_urls = "" # @param {type:"string"}
# @markdown ---
# @markdown - 将制定文件夹填在 `target_dir`，档案的下载地址填在 `file_urls`。
# @markdown - 网址之间用英文逗号隔开。例：`url1`, `url2`, `url3`
file_urls = list(map(str.strip, file_urls.split(",")))

import os
if os.path.exists(os.path.split(target_dir)[0]):
    os.makedirs(target_dir, exist_ok=True)
    %cd {target_dir}
    for url in file_urls:
        if url != '':
            ! wget -nc --content-disposition {url}

In [ ]:
# @title 2.4 自定义资料夹连接
google_drive_dir = "" # @param {type:"string"} 
webui_dir = "" # @param {type:"string"}
# @markdown ---
# @markdown - 将云端硬盘资料夹填在 `google_drive_dir`，WebUI资料夹填在 `webui_dir`。

import os
if os.path.exists(os.path.split(webui_dir)[0]):
    os.makedirs(google_drive_dir, exist_ok=True)
    if os.path.exists(webui_dir):
        ! cp -Rf {webui_dir}/* {google_drive_dir}
    else:
        os.makedirs(webui_dir, exist_ok=True)
    ! rm -Rf {webui_dir} && ln -s {google_drive_dir} {webui_dir}

## 3 - 其他设置

In [ ]:
# @title 3.1 config.json
recommended_settings = True # @param {type:"boolean"}

import json

def read_config(filename):
  with open(filename, "r") as f:
    config = json.load(f)
  return config

def write_config(filename, config):
  with open(filename, "w") as f:
    json.dump(config, f, indent=4)

if recommended_settings == True:
    %cd /content/SDWCT/extensions
    config_file = f"{config_path}/config.json"
    config = read_config(config_file)
    if "CLIP_stop_at_last_layers" not in config or config["CLIP_stop_at_last_layers"] == 1:
        config["CLIP_stop_at_last_layers"] = 2
    if "quicksettings" not in config or config["quicksettings"] == "sd_model_checkpoint":
        config["quicksettings"] = "sd_model_checkpoint, sd_vae"    
    if os.path.exists("sd-webui-controlnet") and ("control_net_max_models_num" not in config or config["control_net_max_models_num"] == 1): 
        config["control_net_max_models_num"] = 3
    if os.path.exists("stable-diffusion-webui-localization-zh_CN"):
        if not os.path.exists("sd-webui-bilingual-localization") and ("localization" not in config or config["localization"] == "None"): 
            config["localization"] = "zh_CN"
        elif os.path.exists("sd-webui-bilingual-localization") and ("bilingual_localization_file" not in config or config["bilingual_localization_file"] == "None"):
            config["localization"] = "None"
            config["bilingual_localization_file"] = "zh_CN"
    if os.path.exists("a1111-sd-webui-tagcomplete"):
       config["tac_tagFile"] = "zh_cn.csv"
       config["tac_extra.extraFile"] = "zh_cn.csv"
    write_config(config_file, config)  

In [ ]:
# @title 3.2 命令行参数
medvram = False # @param {type:"boolean"}
lowram = False # @param {type:"boolean"}
# @markdown SSH Tunnel
ssh_tunnel = "gradio" # @param ["gradio", "ngrok(推荐，需填写ngrok_authtoken)", "cloudflared", "localhost.run", "remote.moe", "googleusercontent.com", "multiple"]
ngrok_authtoken = "" # @param {type:"string"}
# @markdown ---
# @markdown ngrok_authtoken 填写方法
# @markdown - 註冊 [ngrok](https://ngrok.com/) 账号
# @markdown - Your Authtoken > Copy
# @markdown - ssh_tunnel 选 ngrok > 贴上 ngrok_authtoken
# @markdown - 等 local URL 出来之后再点 ngrok URL

import os
from google.colab.output import eval_js

os.environ['colab_url'] = eval_js("google.colab.kernel.proxyPort(7860, {'cache': false})")
os.environ["COMMANDLINE_ARGS"] = "--no-half-vae --xformers \
                  --no-hashing --enable-insecure-extension-access \
                  --disable-safe-unpickle --opt-channelslast --theme dark"

if medvram == True:
    os.environ["COMMANDLINE_ARGS"] += " --medvram"
if lowram == True:
    os.environ["COMMANDLINE_ARGS"] += " --lowram"

if ssh_tunnel == "gradio":
    os.environ["COMMANDLINE_ARGS"] += " --share"
elif ssh_tunnel == "ngrok(推薦，需填寫ngrok_authtoken)":
    os.environ["COMMANDLINE_ARGS"] += (" --ngrok " + ngrok_authtoken)
elif ssh_tunnel == "cloudflared":
    os.environ["COMMANDLINE_ARGS"] += " --share --cloudflared"
elif ssh_tunnel == "localhost.run":
    os.environ["COMMANDLINE_ARGS"] += " --share --localhostrun"
elif ssh_tunnel == "remote.moe":
    os.environ["COMMANDLINE_ARGS"] += " --share --remotemoe"
elif ssh_tunnel == "googleusercontent.com":
    os.environ["COMMANDLINE_ARGS"] += " --share --googleusercontent"
elif ssh_tunnel == "multiple":
    os.environ["COMMANDLINE_ARGS"] += " --multiple"

## 4 - 启动 WebUI

In [ ]:
# @title 4.1 启动 WebUI
# @markdown - 点选 public URL 开启 WebUI<br>
# @markdown ![](https://i.imgur.com/y3xGiIX.png)

%cd /content/SDWCT/
! python launch.py